In [4]:
from django.db.models import Q
from base_app.models import ConfigItems
from leagues.models import Competition, Season
from base_app.decorators import cleanup_selenium_instances,timed_retry
from base_app.helpers import get_name_mappings
from games.models import Game


import pandas as pd
import re
from datetime import datetime
from pprint import pprint


In [5]:
BASE_FILE_DIR = "D:\All Season Data\Files"
BASE_LOGS_DIR = "D:\All Season Data\Logs"

In [6]:
SA_TO_COMP_NAME_MAP = get_name_mappings(source="SA",target="N",source_as_file_names=True,target_as_file_name=False)
FM_TO_COMP_NAME_MAP = get_name_mappings(source="FM",target="N",source_as_file_names=True,target_as_file_name=False)


In [7]:
def filter_file_details(filename):
    """
    Extracts competition, season, and type from a filename using a regex.

    Args:
        filename (str): The name of the file to parse.

    Returns:
        dict: A dictionary containing the extracted details, or None if the
              filename does not match the expected format.
    """
    # Regex to capture competition, season, and type from the filename
    # The pattern matches:
    # 1. A non-greedy group for the competition name (.*?)
    # 2. A specific group for the season, which can be 'YYYY' or 'YYYY_YYYY'
    # 3. A word group for the type (\w+)
    regex = r'^(.*?)_(\d{4}(?:_\d{4})?)_(\w+)\.xlsx$'
    match = re.search(regex, filename)

    if match:
        # Extract the captured groups and clean them up
        competition_name = match.group(1).replace('_', ' ')
        season_value = match.group(2).replace('_', '/')
        type_value = match.group(3)

        return {
            'competition': competition_name,
            'season': season_value,
            'type': type_value
        }
    else:
        return {}

def filter_log_file_details(filename):
    league,season = [x for x in filename.split("_-_")][:2]
    
    return {
            'competition': league.replace("_"," "),
            'season': season.replace("_","/"),
        }
    

In [10]:
def get_all_files(directory_path="D:\All Season Data\Files"):
    files_data = []
    fl = False
    for lvl_1,lvl_2,file_list in os.walk(directory_path,):
        if file_list == []:
            continue
        batch_no = next((x.split("_")[0].capitalize() for x in lvl_1.split("\\") if "_MetaData" in x),"B8")
        confederation,region = [x.strip(" ") for x in lvl_1.split("\\") ][-2:]
        for file_name in file_list :
            df = None
            temp = {}
            temp['batch'] = batch_no
            temp['confederation'] = confederation
            temp['region'] = region
            temp = {**temp,**filter_file_details(file_name)}
            try :
                df = pd.read_excel(f"{lvl_1}/{file_name}")
                temp['matches'] = df.shape[0]
                temp['team_counts'] = len(set(list(set(df['home_team']))+list(set(df['away_team'])))) 
                del df
            except :
                temp['team_counts'] = 0 
                temp['matches'] = 0
                if df:
                    del df
            temp['xl_file_name'] = f"{lvl_1}/{file_name}"
            files_data.append(temp)
    return pd.DataFrame(files_data)
files_df = get_all_files()    

In [11]:
files_df = files_df.sort_values(['region','competition','season','batch'])

In [12]:
files_df.season.unique()

array(['2018', '2019', '2020', '2021', '2022', '2023/2024', '2024/2025',
       '2018/2019', '2019/2020', '2023', '2024', '2017/2018', '2020/2021',
       '2025', '2021/2022', '2022/2023'], dtype=object)

In [13]:
done = []
condesed = []
r = []
for i,row in files_df.iterrows() :
    line = row.to_dict()
    region = row['region']
    competition = row['competition']
    season = row['season']
    s = f'{region.replace(" ","_")}_{competition.replace(" ","_")}_{season.replace("/","_")}'
    if s in done:
        continue
    condition = (
        (files_df['region'] == region) &
        (files_df['season'] == season) &
        (files_df['competition'] == competition)
    )
    filtered = files_df[condition]
    max_val = max(list(filtered['matches']))
    line['File_matches'] = max_val
    line['competition'] = SA_TO_COMP_NAME_MAP.get(f"{region}|{competition.replace('.','')}")   
    if line['competition'] is None :
        line['competition'] = FM_TO_COMP_NAME_MAP.get(f"{region}|{competition.replace('.','')}")   
    done.append(s)
    condesed.append(line)


In [14]:
condensed_file_df = pd.DataFrame(condesed)
condensed_file_df = condensed_file_df.sort_values(['region','competition','season','batch'])

In [15]:
def parse_and_look_at_errors(file_path) :
    with open(file_path,"r",encoding='utf-8') as f :
        all_logs = [f"2025{x}".replace("\n"," ").replace("\t"," ").replace('[','').replace(']','') for x in f.read().split("\n2025")]
    err_cnts = {
        "Total Error Count" : 0,        
        "TimeoutException" : 0,
        "ReadTimeoutError" : 0,
        "InvalidSessionIdException" : 0,
        "NoSuchWindowException" : 0,
        "AttributeError" : 0,
        "ValueError" : 0,
        "NoSuchElementException" : 0,
        "HTTPConnectionPool" : 0
    }
    for i,x in enumerate(all_logs) :
        if ('Error' in x or 'Exception' in x.lower()) and 'Cleanup failed' not in x:
           err_cnts["Total Error Count"] += 1
        for k,v in err_cnts.items() :
            if k in x :
                err_cnts[k] = v+1
    return err_cnts

def parse_and_look_fixture_count(file_path) :
    with open(file_path,"r",encoding='utf-8') as f :
        all_logs = [f"2025{x}".replace("\n"," ").replace("\t"," ").replace('[','').replace(']','') for x in f.read().split("\n2025")]
    count_logs = [x for x in all_logs if 'Found a total of ' in x and ' matches' in x ]
    # print(file_path.split('/')[-1],count_logs)
    for x in all_logs :
        match = re.search(r'Found a total of (\d+) matches', x)
    
        if match:
            match_count = int(match.group(1))
            return match_count
    return None
def get_all_logs(directory_path="D:\All Season Data\Logs"):
    log_data = []
    all_errs = []
    done_list = []
    for lvl_1,lvl_2,file_list in os.walk(directory_path,):
        if file_list == []:
            continue
        batch_no = next((x.split("_")[0].capitalize() for x in lvl_1.split("\\") if "_runtime" in x),"B6")
        confederation,region = [x.strip(" ") for x in lvl_1.split("\\") ][-2:]
        for file_name in file_list :
            # num = file_name.split("_")[-1]
            # proc_num  = int(num) if num.isdigit() else "SHINSHINAKIBABLABU"
            # file_name = file_name.replace(f"_{proc_num}","")
            
            # if file_name in done_list :
            #     print("no need its done : ",file_name)
            #     continue
            temp = {}
            temp['batch'] = batch_no
            temp['confederation'] = confederation
            temp['region'] = region
            temp = {**temp,**filter_log_file_details(file_name)}
            temp['fixture_count'] = parse_and_look_fixture_count(f"{lvl_1}/{file_name}")
            # temp = {**temp,**parse_and_look_at_errors(f"{lvl_1}/{file_name}")}
            comp_obj = Competition.objects.get(competition_name=temp['competition'])
            temp['confederation']= comp_obj.confederation
            temp['region'] = comp_obj.country
            temp['competition'] = comp_obj.name_scoresaway.replace(".","")
            temp['competition_alt'] = comp_obj.competition_name.replace(".","")
            temp['lg_file_name'] = f"{lvl_1}/{file_name}"
            log_data.append(temp)
            # done_list.append(file_name)
    return pd.DataFrame(log_data)
    # return pd.DataFrame(logs_data)
err_df = get_all_logs()

In [16]:
err_df = err_df.dropna(subset=['fixture_count']).sort_values(['region','competition','season','batch'])

In [17]:
done = []
condesed = []
for i,row in err_df.iterrows() :
    line = row.to_dict()
    region = row['region']
    competition = row['competition']
    season = row['season']
    s = f'{region.replace(" ","_")}_{competition.replace(" ","_")}_{season.replace("/","_")}'
    if s in done:
        continue
    condition = (
        (err_df['region'] == region) &
        (err_df['season'] == season) &
        (err_df['competition'] == competition)
    )
    filtered = err_df[condition]
    max_val = max(list(filtered['fixture_count']))
    line['Log_Matches'] = max_val
    line['competition'] = SA_TO_COMP_NAME_MAP[f"{region}|{competition.replace('.','')}"]
    done.append(s)
    # print(f"{s} | {max_val}")
    condesed.append(line)

condensed_err_df = pd.DataFrame(condesed)
condensed_err_df = condensed_err_df.sort_values(['region','competition','season','batch'])
condensed_err_df

,batch,confederation,region,competition,season,fixture_count,competition_alt,lg_file_name,Log_Matches
0,B1,AFC,AFC,AFC Champions League,2018,140.0,AFC Champions League,D:\All Season Data\Logs\b1_runtime_logs/AFC_Ch...,140.0
1,B1,AFC,AFC,AFC Champions League,2019,145.0,AFC Champions League,D:\All Season Data\Logs\b1_runtime_logs/AFC_Ch...,145.0
2,B1,AFC,AFC,AFC Champions League,2020,113.0,AFC Champions League,D:\All Season Data\Logs\b1_runtime_logs/AFC_Ch...,113.0
3,B1,AFC,AFC,AFC Champions League,2021,140.0,AFC Champions League,D:\All Season Data\Logs\b1_runtime_logs/AFC_Ch...,140.0
4,A1,AFC,AFC,AFC Champions League,2022,137.0,AFC Champions League,D:\All Season Data\Logs\a1_runtime_logs/AFC_Ch...,137.0
...,...,...,...,...,...,...,...,...,...
543,B1,CONMEBOL,Venezuela,Primera Division,2021,303.0,Primera Division,D:\All Season Data\Logs\b1_runtime_logs/Primer...,303.0
544,A1,CONMEBOL,Venezuela,Primera Division,2022,277.0,Primera Division,D:\All Season Data\Logs\a1_runtime_logs/Primer...,277.0
545,A1,CONMEBOL,Venezuela,Primera Division,2023,222.0,Primera Division,D:\All Season Data\Logs\a1_runtime_logs/Primer...,222.0
546,B1,CONMEBOL,Venezuela,Primera Division,2024,234.0,Primera Division,D:\All Season Data\Logs\b1_runtime_logs/Primer...,234.0


In [18]:
condensed_file_df

,batch,confederation,region,competition,season,type,matches,team_counts,xl_file_name,File_matches
0,B1,AFC,AFC,AFC Champions League,2018,events,140,46,D:\All Season Data\Files\b1_MetaData\AFC\AFC/A...,140
1,B1,AFC,AFC,AFC Champions League,2019,events,145,51,D:\All Season Data\Files\b1_MetaData\AFC\AFC/A...,145
2,B1,AFC,AFC,AFC Champions League,2020,events,113,49,D:\All Season Data\Files\b1_MetaData\AFC\AFC/A...,113
3,B1,AFC,AFC,AFC Champions League,2021,events,140,45,D:\All Season Data\Files\b1_MetaData\AFC\AFC/A...,140
4,B1,AFC,AFC,AFC Champions League,2022,events,127,46,D:\All Season Data\Files\b1_MetaData\AFC\AFC/A...,137
...,...,...,...,...,...,...,...,...,...,...
596,B1,CONMEBOL,Venezuela,Primera Division,2021,events,303,21,D:\All Season Data\Files\b1_MetaData\CONMEBOL\...,303
597,B2,CONMEBOL,Venezuela,Primera Division,2022,events,240,16,D:\All Season Data\Files\b2_MetaData\CONMEBOL\...,277
598,B2,CONMEBOL,Venezuela,Primera Division,2023,events,207,15,D:\All Season Data\Files\b2_MetaData\CONMEBOL\...,222
599,B2,CONMEBOL,Venezuela,Primera Division,2024,events,234,14,D:\All Season Data\Files\b2_MetaData\CONMEBOL\...,234


In [19]:
f_comps = set(condensed_file_df['competition'].str.replace('.',''))
l_comps = set(err_df['competition'].str.replace('.',''))

In [20]:
merged_df = pd.merge(condensed_file_df,condensed_err_df,how='left',on=['confederation','region','competition','season']).dropna()

In [21]:
merged_df['updown'] = abs(merged_df['File_matches'] - merged_df['Log_Matches'])
merged_df[(merged_df['type'] == 'events') & (merged_df['updown'] > 0)][["confederation","region","competition","season","type","updown"]].sort_values('updown',ascending=False).to_excel("Batch_3_Run.xlsx",index=False)